In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("datamart_test.db")

In [3]:
cur = con.cursor()

In [7]:
cur.execute("drop table if exists customers")
cur.execute("drop table if exists orders")
cur.execute("drop table if exists order_details")

In [8]:
cur.execute('''CREATE TABLE customers(
            customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            email TEXT,
            region TEXT 
            )''')

In [9]:
cur.execute('''CREATE TABLE orders(
                order_id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER,
                order_date TEXT,            
                shipping_priority TEXT,
                FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
            )
        ''')

In [10]:
cur.execute('''create table order_details(
            order_detail_id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER,
            product_id INTEGER,
            quantity INTEGER,
            price FLOAT,
            discount FLOAT,
            FOREIGN KEY (order_id) references orders(order_id)
            )''')

In [11]:
#Se agregan valores a la tabla customers
cur.execute('''
        insert into customers ("name","email","region") values ("John Doe", "John@gmail.com", "North")
''')
cur.execute('''
        insert into customers ("name","email","region") values ("Juan Perez", "JuanPerez@hotmail.com", "East")
''')
cur.execute('''
        insert into customers ("name","email","region") values ("Jonathan Múnera", "JD@gmail.com", "West")
''')


In [12]:
#Se agregan valores a la tabla orders
cur.execute('''
        insert into orders ("customer_id","order_date","shipping_priority") values ("3", "2023-03-31", "High")
''')
cur.execute('''
        insert into orders ("customer_id","order_date","shipping_priority") values ("1", "2023-12-12", "Low")
''')
cur.execute('''
        insert into orders ("customer_id","order_date","shipping_priority") values ("3", "2024-03-31", "High")
''')

In [13]:
cur.execute('''
        insert into order_details ("order_id", "product_id", "quantity", "price", "discount") values ("1", "15", "2", "70000", ".3")
''')
cur.execute('''
        insert into order_details ("order_id","product_id","quantity","price","discount") values ("2", "45", "20", "20000", "0")
''')
cur.execute('''
        insert into order_details ("order_id","product_id","quantity","price","discount") values ("3", "15", "3", "120000", ".2")
''')

In [14]:
#Ingreso total por cliente
cur.execute('''Select customers.customer_id, sum(price)
            from order_details
            inner join orders
            on orders.order_id = order_details.order_id
            right join customers
            on customers.customer_id = orders.customer_id
            group by customers.customer_id
''').fetchall()

[(1, 20000.0), (2, None), (3, 190000.0)]

In [15]:
#Producto más vendido por región
cur.execute('''Select customers.region, product_id, price*quantity
            from order_details
            inner join orders
            on orders.order_id = order_details.order_id
            right join customers
            on customers.customer_id = orders.customer_id
            group by customers.region, product_id
''').fetchall()

[('East', None, None), ('North', 45, 400000.0), ('West', 15, 140000.0)]

In [16]:
#Ingreso promedio por cliente y region al mes
cur.execute('''
            Select customers.customer_id,
                    customers.region,
                    substring(replace(date(order_date),"-",""), 1, 6) as mes,
                    sum(order_details.price)
            from customers 
            left join orders
            on orders.customer_id = customers.customer_id
            inner join order_details
            on orders.order_id = order_details.order_id
            group by customers.customer_id, region, mes
''').fetchall()

[(1, 'North', '202312', 20000.0),
 (3, 'West', '202303', 70000.0),
 (3, 'West', '202403', 120000.0)]

In [17]:
#Top 5 clientes en el último año
cur.execute('''
    Select  customers.customer_id,
            count(order_id) as num_orders
            from customers
            inner join orders
            on customers.customer_id = orders.customer_id            
            where substring(order_date,1,4) = cast(substring(date("now"),1,4)-1 as string)
            group by customers.customer_id
            order by num_orders desc
            limit 5
''').fetchall()

[(3, 1)]